## Imports

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## LSUV

In [3]:
#export
from exp.nb_07 import *

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [5]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [6]:
nfs = [8,16,32,64,64]
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
#     @property
#     def bias(self): return -self.relu.sub
#     @bias.setter
#     def bias(self,v): self.relu.sub = -v
#     @property
#     def weight(self): return self.conv.weight

In [7]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

In [8]:
Learner.children = lambda x: list(x.model.children())

In [11]:
learn.children()[0].

ConvLayer(
  (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu): GeneralRelu()
)

In [31]:
[1,2]+[2,3, 4], [1,2]+[[2,3, 4]], [[1,2]]+[[2,3, 4]]

([1, 2, 2, 3, 4], [1, 2, [2, 3, 4]], [[1, 2], [2, 3, 4]])

In [38]:
[1,2] + 3

TypeError: can only concatenate list (not "int") to list

In [32]:
sum([1,2],[2,3,4])

TypeError: can only concatenate list (not "int") to list

In [33]:
sum([1,2], [[2,3, 4]])

TypeError: can only concatenate list (not "int") to list

In [2]:
sum([[1,2], [2,3]],[2,3, 4])

[2, 3, 4, 1, 2, 2, 3]

In [39]:
sum([[[[1,2]]]],[2,3, 4])

[2, 3, 4, [[1, 2]]]

In [35]:
sum([1,2,4,4], 19)

30

In [ ]:
def lin_chec(m, cond):
    if cond(m):
        return [m]
    [lin_chec()

In [ ]:
sum()

In [73]:
def find_lin_layers(m):
    if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, GeneralRelu)):
        print('insintance',m)
        return [m]
    print('afterisinatnce', m)
    return sum([find_lin_layers(o) for o in m.children()], [])

In [63]:
m1 = nn.Sequential(nn.Linear(12, 24), nn.Linear(24, 32), nn.Linear(32, 5))

In [68]:
m1[0]

Linear(in_features=12, out_features=24, bias=True)

In [74]:
find_lin_layers(m1)

afterisinatnce Sequential(
  (0): Linear(in_features=12, out_features=24, bias=True)
  (1): Linear(in_features=24, out_features=32, bias=True)
  (2): Linear(in_features=32, out_features=5, bias=True)
)
insintance Linear(in_features=12, out_features=24, bias=True)
insintance Linear(in_features=24, out_features=32, bias=True)
insintance Linear(in_features=32, out_features=5, bias=True)


[Linear(in_features=12, out_features=24, bias=True),
 Linear(in_features=24, out_features=32, bias=True),
 Linear(in_features=32, out_features=5, bias=True)]

In [54]:
t=find_lin_layers(learn.model)

In [56]:
t

[Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
 GeneralRelu(),
 Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 GeneralRelu(),
 Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 GeneralRelu(),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 GeneralRelu(),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 GeneralRelu(),
 Linear(in_features=64, out_features=10, bias=True)]

In [95]:
def find(m):
    for c in m.children():
        print('checl')
        if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, GeneralRelu)):
            print('insintance',m)
            return [m]
        find(c)

In [96]:
[find(c) for c in learn.model.children()]

checl
checl
checl
checl
checl
checl
checl
checl
checl
checl


[None, None, None, None, None, None, None, None]

In [89]:
learn.children()

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), AdaptiveAvgPool2d(output_size=1), Lambda(), Linear(in_features=64, out_features=10, bias=True)]

In [88]:
list(learn.children()[0].children())

[Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
 GeneralRelu()]